In [ ]:
# Cell 1: Import required libraries
import tarfile
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import os

# Cell 2: Set up paths and variables
# Your deployment package directory (based on your folder structure)
deployment_package_dir = '/home/sagemaker-user/keyword-recommendation/sample_deployment/deployment_package'

# S3 bucket details
bucket_name = 'your-s3-bucket-name'  # Replace with your actual bucket name
s3_key = 'models/fin-keyword-model/model.tar.gz'

# SageMaker endpoint name
endpoint_name = 'fin-keyword-endpoint'

# Cell 3: Create the model.tar.gz package
print("Creating model.tar.gz package...")

# Change to the directory containing deployment_package
os.chdir('/home/sagemaker-user/keyword-recommendation/sample_deployment')

# Create the model.tar.gz file
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    # Add the entire deployment_package directory
    tar.add('deployment_package', arcname='.')
    
print("Model package created: model.tar.gz")

# Verify the contents
with tarfile.open('model.tar.gz', 'r:gz') as tar:
    print("Contents of model.tar.gz:")
    for member in tar.getmembers():
        print(f"  {member.name}")

# Cell 4: Upload to S3
print("Uploading model to S3...")

# Initialize S3 client
s3 = boto3.client('s3')

# Upload the model package
s3.upload_file('model.tar.gz', bucket_name, s3_key)

print(f"Model uploaded to s3://{bucket_name}/{s3_key}")

# Cell 5: Initialize SageMaker session
print("Initializing SageMaker session...")

sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name

print(f"Using region: {region}")
print(f"Using role: {role}")

# Cell 6: Deploy the model
print("Deploying model to SageMaker...")

# Create the model
model = PyTorchModel(
    model_data=f's3://{bucket_name}/{s3_key}',
    role=role,
    framework_version='1.9.0',  # PyTorch version
    py_version='py38',         # Python version
    entry_point='inference.py',
    name='fin-keyword-model',
    sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

print(f"Endpoint deployed: {predictor.endpoint_name}")

# Cell 7: Test the endpoint
print("Testing the endpoint...")

# Test data
test_data = [
    {
        "description": "steel pipe 2 inch diameter",
        "keyword": "pipe"
    },
    {
        "description": "aluminum sheet 5mm thickness", 
        "keyword": "sheet"
    }
]

# Make prediction
try:
    response = predictor.predict(test_data)
    print("Test successful!")
    print("Response:", response)
except Exception as e:
    print("Error:", str(e))

# Cell 8: Monitor the deployment
print("Monitoring deployment status...")

# Check endpoint status
sm_client = boto3.client('sagemaker')
endpoint_status = sm_client.describe_endpoint(
    EndpointName=endpoint_name
)['EndpointStatus']

print("Endpoint status:", endpoint_status)

# Cell 9: Save endpoint information for later use
endpoint_info = {
    'endpoint_name': endpoint_name,
    'region': region,
    'model_data': f's3://{bucket_name}/{s3_key}'
}

print("Endpoint information:")
print(endpoint_info)

# Cell 10: Create a cleanup function (optional)
def cleanup_deployment():
    """Delete the endpoint and model to avoid ongoing costs"""
    try:
        predictor.delete_model()
        predictor.delete_endpoint()
        print("Endpoint and model deleted successfully")
    except Exception as e:
        print(f"Error during cleanup: {str(e)}")

# Uncomment the line below if you want to cleanup immediately
# cleanup_deployment()